In [11]:
import pandas as pd
import numpy as np
from plan import *
from testing import generate_all_single

In [10]:
data = Dataset()

In [5]:
output_base_path = "F:\SHRINIVAS\KGP\BTP\Meal-Planning\Outputs\single_objective\\"

In [6]:
df = pd.read_csv(output_base_path+"Weights1.tsv",sep='\t')

In [7]:
df.head()

,Dish 1 id,Dish 2 id,Dish 3 id,Dish 4 id,Dish 5 id,Dish 6 id,Dish 7 id,Dish 8 id,Dish 9 id,Dish 10 id,Dish 11 id,Dish 12 id,Dish 13 id,Dish 14 id,Dish 15 ID
0,6070,0,0,778,5079,509,0,0,605,0,1727,700,4014,0,0
1,6070,0,0,778,5079,509,0,0,2842,0,1727,700,4014,0,0
2,468,0,0,778,5079,509,0,0,2666,0,1727,700,4014,0,0
3,5851,0,0,778,5079,509,0,0,2666,0,1727,700,4014,0,0
4,6070,0,0,778,5079,509,0,0,2949,0,1727,700,4014,0,0


In [8]:
all_data = df.values

In [12]:
all_single = generate_all_single()

In [77]:
def get_meal_plans(weights_file_name,all_single):
    df = pd.read_csv(output_base_path+f"{weights_file_name}.tsv",sep='\t')
    all_data = df.values
    meal_plans: list[MealPlan] = []
    for plan_id,plan in enumerate(all_data):
        meal_plan = []
        for dish_id in plan:
            meal = "Breakfast"
            if dish_id > 2 and dish_id <= 7:
                meal = "Lunch"
            elif dish_id > 7 and dish_id <= 9:
                meal = "Snacks"
            elif dish_id > 9:
                meal = "Dinner"
            if dish_id == 0:
                dish = Dish.get_padding_dish()
            else:
                dish = Dish(
                    id = str(dish_id),
                    quantity = 1,
                    vector = data.get_dish_vector(dish_id),
                    title = data.get_dish_title(dish_id),
                    meal = meal,
                    cuisine = data.get_dish_cuisine(dish_id),
                    tags = data.get_dish_cuisine(dish_id),
                    category = data.get_dish_category(dish_id)
                )
            meal_plan.append(dish)
        problem_config=ProblemConfig(**all_single[plan_id//5])
        meal_plans.append(MealPlan(problem_config,data,meal_plan))
    return meal_plans

Cuisine Factor

In [16]:
cuisines=['North America', 'United States', 'Europe', 'Italy',
            'Middle East', 'South East Asia', 'Canada', 'France', 'Mexico',
            'British Isles', 'Australia & NZ', 'Greece', 'Eastern Europe',
            'Asia', 'South America', 'China', 'Japan', 'Africa',
            'Indian Subcontinent', 'Korea']

In [17]:
cuisine_matrix = np.zeros((len(cuisines),len(cuisines)))

In [48]:
one_dist = [
    ["North America","United States"],
    ["North America","Canada"],
    ["Europe","Italy"],
    ["Europe","France"],
    ["Europe","Mexico"],
    ["Europe","British Isles"],
    ["Europe", "Greece"],
    ["Europe","Eastern Europe"],
    ["Asia","Middle East"],
    ["Asia","China"],
    ["Asia","Japan"],
    ["Asia","South East Asia"],
    ["Asia","Korea"]
]

two_dist = [
    ["United States","Canada"],
    ["Italy","France"],
    ["Greece","Eastern Europe"],
]

three_dist = [
    ["Japan","China"],
    ["China","Korea"],
    ["Japan","Korea"]
]

In [49]:
for cid1, cuisine1 in enumerate(cuisines):
    for cid2, cuisine2 in enumerate(cuisines):
        if cid1 == cid2:
            cuisine_matrix[cid1][cid2]=0
        elif [cuisine1,cuisine2] in one_dist or [cuisine2,cuisine1] in one_dist:
            cuisine_matrix[cid1][cid2]=1
            cuisine_matrix[cid2][cid1]=1
        elif [cuisine1,cuisine2] in two_dist or [cuisine2,cuisine1] in two_dist:
            cuisine_matrix[cid1][cid2]=2
            cuisine_matrix[cid2][cid1]=2
        elif [cuisine1,cuisine2] in three_dist or [cuisine2,cuisine1] in three_dist:
            cuisine_matrix[cid1][cid2]=3
            cuisine_matrix[cid2][cid1]=3
        else:
            cuisine_matrix[cid1][cid2]=5
            cuisine_matrix[cid2][cid1]=5

In [50]:
def get_cuisine_score(meal_plan:MealPlan):
    cnt = 0 
    value = 0
    for id1 in range(0,len(meal_plan.plan)):
        if meal_plan.plan[id1].id == 0:
            continue
        for id2 in range(id1 + 1, len(meal_plan.plan)):
            if meal_plan.plan[id2].id == 0 or meal_plan.plan[id1].meal != meal_plan.plan[id2].meal:
                continue
            if meal_plan.plan[id1].cuisine==-1 or meal_plan.plan[id2].cuisine==-1:
                continue
            cnt += 1
            cid1 = cuisines.index(meal_plan.plan[id1].cuisine)
            cid2 = cuisines.index(meal_plan.plan[id2].cuisine)
            value += cuisine_matrix[cid1][cid2] / 5
    if cnt == 0:
        return 0
    return value / cnt

Course Constraint

In [22]:
df_dishes=pd.read_csv("./Data/Processed/dishes.csv")

In [92]:
df_dishes['Category'].unique()

array(['appetizer', 'dessert', 'main-dish'], dtype=object)

In [31]:
def get_course_scores(meal_plan:MealPlan):
    breakfast_dishes = []
    lunch_dishes = []
    snacks_dishes = []
    dinner_dishes = []
    for dish_id, dish in enumerate(meal_plan):
        if dish_id <= 2:
            breakfast_dishes.append(dish.category)
        elif dish_id <= 7:
            lunch_dishes.append(dish.category)
        elif dish_id <= 9:
            snacks_dishes.append(dish.category)
        else:
            dinner_dishes.append(dish.category)
    breakfast_score = 0
    if ('appetizer' in breakfast_dishes or 'dessert' in breakfast_dishes) and 'main-dish' not in breakfast_dishes:
        breakfast_score = 1
    lunch_score = 0
    if 'appetizer' in lunch_dishes and ('main-dish' in lunch_dishes or len([val for val in lunch_dishes if val=='appetizer'])==2):
        lunch_score = 1
    snacks_score = 0
    if ('appetizer' in snacks_dishes or 'dessert' in snacks_dishes) and 'main-dish' not in snacks_dishes:
        snacks_score = 1
    dinner_score = 0
    if 'appetizer' in dinner_dishes and ('main-dish' in dinner_dishes or len([val for val in dinner_dishes if val=='appetizer'])==2):
        dinner_score = 1
    overall_score = (breakfast_score + lunch_score + snacks_score + dinner_score ) / 4

    return {
        "breakfast": breakfast_score,
        "lunch": lunch_score,
        "snacks": snacks_score,
        "dinner": dinner_score,
        "overall": overall_score
    }

In [88]:
def evaluate_meal_plans(meal_plans):
    avg_res = {
        "diversity": 0,
        "combination": 0,
        "preference": 0,
        "cuisine": 0,
        "breakfast": 0,
        "lunch": 0,
        "snacks": 0,
        "dinner": 0,
        "overall": 0
    }
    for meal_plan in meal_plans:
        avg_res["diversity"]+=meal_plan.get_diversity()
        avg_res["combination"]+=meal_plan.get_combi_value()
        avg_res["preference"]+=meal_plan.get_pos_preference() - meal_plan.get_neg_preference()
        avg_res["cuisine"]+=get_cuisine_score(meal_plan)
        course_scores = get_course_scores(meal_plan.plan)
        avg_res["breakfast"]+=course_scores["breakfast"]
        avg_res["lunch"]+=course_scores["lunch"]
        avg_res["snacks"]+=course_scores["snacks"]
        avg_res["dinner"]+=course_scores["dinner"]
        avg_res["overall"]+=course_scores["overall"]

    for k,v in avg_res.items():
        avg_res[k]/=len(meal_plans)
    return avg_res

In [89]:
res = []
for id in range(1,9):
    meal_plans = get_meal_plans(f"Weights{id}",all_single)
    avg_res = {
        "title": f"Weights{id}",
    }
    avg_res.update(evaluate_meal_plans(meal_plans))
    res.append(avg_res)

In [90]:
result_df = pd.DataFrame(res)

In [91]:
result_df.head(10)

,title,diversity,combination,preference,cuisine,breakfast,lunch,snacks,dinner,overall
0,Weights1,0.658937,0.674382,0.719439,0.385020,0.752000,0.720000,0.988000,0.820000,0.820000
1,Weights2,0.639977,0.674876,0.523882,0.450944,0.804878,0.756098,0.979675,0.650407,0.797764
2,Weights3,0.698475,0.661646,0.634848,0.464634,0.804444,0.777778,1.000000,0.777778,0.840000
3,Weights4,0.655237,0.640285,0.851197,0.265322,0.792000,0.800000,1.000000,0.760000,0.838000
4,Weights5,0.675123,0.655201,0.603977,0.488247,0.843478,0.847826,1.000000,0.826087,0.879348
5,Weights6,0.625849,0.683657,0.225470,0.425868,0.802752,0.651376,0.995413,0.770642,0.805046
6,Weights7,0.685787,0.610501,0.702909,0.338878,0.747967,0.776423,0.991870,0.756098,0.818089
7,Weights8,0.628819,0.698366,0.358573,0.309426,0.733668,0.698492,1.000000,0.633166,0.766332
